# Wannierization using Wannier90

DFTK features an interface with the program
[Wannier90](http://www.wannier.org/),
in order to compute maximally-localized Wannier functions (MLWFs)
from an initial self consistent field calculation.
All processes are handled by calling the routine `run_wannier90`.

This example shows how to obtain the MLWFs corresponding
to the first eight bands of silicon. Since the bands 5 to 8 are entangled,
12 bands are first computed to obtain 8 MLWFs by a disantanglement procedure.

In [1]:
using DFTK

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/pbe/Si-q4"))
atoms = [ Si => [zeros(3), 0.25*[-1, 3, -1]] ]  # Non-symmetric silicon crystal
model = model_PBE(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])

scfres = self_consistent_field(basis, tol=1e-12, n_bands=12);

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -7.873756548036         NaN   1.98e-01   0.80    6.3
  2   -7.876459030172   -2.70e-03   3.13e-02   0.80    1.0
  3   -7.876572933246   -1.14e-04   3.22e-03   0.80    5.1
  4   -7.876573866521   -9.33e-07   3.27e-04   0.80    3.4
  5   -7.876573877339   -1.08e-08   1.82e-05   0.80    4.0
  6   -7.876573877477   -1.37e-10   3.60e-06   0.80    5.0
  7   -7.876573877487   -1.06e-11   6.24e-07   0.80    4.3
  8   -7.876573877488   -2.11e-13   7.18e-08   0.80    4.3


Now we use the `run_wannier90` routine to generate all files needed by
wannier90 and to perform the wannierization procedure.
In Wannier90's convention, all files are named with the same prefix and only differ by
their extensions. By default all generated input and output files are stored
in the subfolder "wannier90" under the prefix "wannier" (i.e. "wannier90/wannier.win",
"wannier90/wannier.wout", etc.). A different file prefix can be given with the
keyword argument `fileprefix` as shown below.

We now solve for 8 MLWF using wannier90:

In [2]:
using wannier90_jll  # Needed to make run_wannier90 available
run_wannier90(scfres;
              fileprefix="wannier/Si",
              n_wannier=8,
              num_print_cycles=50,
              num_iter=500,
              dis_win_max=17.185257,
              dis_froz_max=6.8318033,
              dis_num_iter=120,
              dis_mix_ratio=1.0)

"wannier/Si"

As can be observed standard optional arguments for the disentanglement
can be passed directly to `run_wannier90` as keyword arguments.

delete temporary files:

In [3]:
rm("wannier", recursive=true)